![Py4Eng](img/logo.png)

# Automation
## Yoav Ram

From the book [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/) by Al Sweigart:

> If you've ever spent hours renaming files or updating hundreds of spreadsheet cells, you know how tedious tasks like these can be. But what if you could have your computer do them for you?

## Monitoring file system events

Monitoring file system events can be valuable for various reasons, particularly in scenarios where you need to track changes, updates, or activities within the file system

We will use the [Watchdog](https://pythonhosted.org/watchdog) package (`pip install watchdog`).

We will write an event handler for filesystem events, and give it to an observer that will use the event handler to handle events on a specific path, the `img` folder.

Our event handler will be very simple, it will just print the filename related to each event.

In [1]:
from watchdog.observers import Observer
import watchdog.events

In [3]:
class MyEventHandler(watchdog.events.FileSystemEventHandler):
    def on_any_event(self, event):
        fname = event.src_path
        print("Something happened to", fname, event.event_type)

In [6]:
path = '../data'
event_handler = MyEventHandler()
observer = Observer()
observer.schedule(event_handler, path, recursive=False)
observer.start()

When we started the observer, it created a new thread for it to run in.

Here we use Jupyter magic to write to a file in that observed path.

In [7]:
%%file ../data/tmp.txt
this is a tmp file

Writing ../data/tmp.txt
Something happened to /Users/yoavram/Work/Teaching/DataSciPy/data/tmp.txt created
Something happened to /Users/yoavram/Work/Teaching/DataSciPy/data modified
Something happened to /Users/yoavram/Work/Teaching/DataSciPy/data/tmp.txt modified


Now we can stop our observer and wait for it to finish whatever it's doing.

In [8]:
observer.stop()
observer.join()

Let's create an observer that converts an image file to PNG format when added to a certain directory.

In [24]:
from pathlib import Path
from PIL import Image # imported for image conversion

class ImageFileHandler(watchdog.events.FileSystemEventHandler):
    def on_created(self, event):
        fname = event.src_path
        print(fname, event.event_type)
        try:            
            with Image.open(fname) as img:
                output_fname = fname[:-4] + ".png"
                img.save(output_fname)
                print(f"Converted '{fname}' to '{output_fname}'")            
        except Exception as e:
            print(f"Could not convert '{fname}': {e}")
        

In [25]:
event_handler = ImageFileHandler()
image_observer = Observer()

image_directory_path = "../data/to_png"
image_observer.schedule(event_handler, image_directory_path, recursive=False)
image_observer.start()

/Users/yoavram/Work/Teaching/DataSciPy/data/to_png/Kobe_Bryant_2014.jpg created
Converted '/Users/yoavram/Work/Teaching/DataSciPy/data/to_png/Kobe_Bryant_2014.jpg' to '/Users/yoavram/Work/Teaching/DataSciPy/data/to_png/Kobe_Bryant_2014.png'


In [26]:
image_observer.stop()
image_observer.join()

## Scheduling jobs

In Python, you can schedule jobs using various libraries and modules.

We will use the [Advanced Python Scheduler](https://apscheduler.readthedocs.org/) (`pip install apscheduler`).
It provides a simple interface for scheduling jobs to run at specific intervals. 

We create a background scheduler (which runs in the background) and start it (in its own thread).
We will discuss threads more extensivley in the coming lectures.

Here's a basic example: we schedule a job to print a message every minute.

In [1]:
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime

In [2]:
scheduler = BackgroundScheduler()
scheduler.start()

Mon Jun 24 16:04:43 2024: Hello!
Mon Jun 24 16:05:11 2024: Hello!
Mon Jun 24 16:05:26 2024: Hello!


Now we write a function that performs some specific job, and add it to the scheduler.

In [3]:
def job():
    print('{time}: Hello!'.format(time=datetime.now().ctime()))
scheduler.add_job(job) # this will run the job now (immediately).

<Job (id=c60a462f13d54f1587cb081291e3019a name=job)>

Now we add another job, but this time we ask that it will run every 15 seconds rather then now.

In [4]:
scheduler.add_job(job, trigger='interval', minutes=0.25) # 15sec = 0.25*1minute
print(datetime.now().ctime())

Mon Jun 24 16:04:56 2024


In [5]:
scheduler.shutdown()

Example - gzip every night at 00:00 compress all images that were converted by watchdog during the day.

In [7]:
import shutil # import this to create compressed archives.

def compress_directory(directory):
    dir_path = directory
    zip_filename = f"images_{datetime.now().strftime('%Y-%m-%d')}"
    shutil.make_archive(zip_filename, 'zip', dir_path)
    print("Directory compressed to", zip_filename)

directory_to_compress = "../data/to_png"

scheduler = BackgroundScheduler()
scheduler.add_job(compress_directory, 'cron', args=[directory_to_compress], hour=0, minute=0)
scheduler.start()

print("Scheduled to compress directory", directory_to_compress, "every night at midnight.")

Scheduled to compress directory ../data/to_png every night at midnight.


Finally we can shutdown the scheduler.

In [8]:
scheduler.shutdown()

# Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)